<a href="https://colab.research.google.com/github/vinitvshah/AI/blob/main/Attention_all_you_Need_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 53.2 MB/s e

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [3]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [5]:
reader = PdfReader('Attention_all_you_need_LLM.pdf')

In [ ]:
reader

In [6]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [7]:
raw_text[:100]

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and'

In [8]:
# spliting the text to read into smaller chunks so that during information retreival we don't hit the token size limits.

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [9]:
len(texts)

50

In [10]:
texts[0]

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and con

In [11]:
texts[1]

'mechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-\nto-German translation task, improving over the existing best results, including\nensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,\nour model establishes a new single-model state-of-the-art BLEU score of 41.8 after\ntraining for 3.5 days on eight GPUs, a small fraction of the training costs of the\nbest models from the literature. We show that the Transformer generalizes well to\nother tasks by applying it successfully to English constituency parsing both with\nlarge and limited training data.'

In [12]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [13]:
docsearch = FAISS.from_texts(texts, embeddings)

In [14]:
docsearch

In [15]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [16]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [17]:
chain_MR = load_qa_chain(OpenAI(), chain_type="map_reduce")

In [18]:
query = "What are the key insights from this document?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' This document discusses the use of attention mechanisms in neural networks, such as the encoder self-attention and the mixture-of-experts layer, to improve the accuracy and interpretability of machine translation systems. Additionally, it provides examples of how attention heads can learn to perform different tasks, and how they can attend to long-distance dependencies.'

In [19]:
query = "how are query, key and value used in transformer?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Query, key and value are used in the transformer to compute the Scaled Dot-Product Attention. The query and key are vectors of dimension dk, while the value is a vector of dimension dv. The output is computed as a weighted sum of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key.'

In [ ]:
summary

' This report provides financial performance data for 1Q23, 4Q22, and 1Q22 for JPMorgan Chase & Co. It includes a balance sheet with average loans of $1.1T, average deposits of $2.3T, CET1 capital of $227B, and a standardized CET1 capital ratio of 13.8%. Net income was $12.6B, $1.6B, and $4.3B respectively. Consumer & Community Banking saw a net income of $5.2B, up 80% YoY, and Corporate & Investment Bank (CIB) reported a net income of $4.4B, up 1% YoY. Asset & Wealth Management reported net income of $1.4B, up 36% YoY, and Corporate reported revenue of $985 million, up $1.9 billion year-over-year. JPMorgan Chase & Co. expects net interest income and net interest income excluding Markets to be around $81 billion in FY2023. Non-GAAP financial measures used by management to assess the performance of the firm are also discussed.'

In [ ]:
query = "What contributed to the growth of net income for Consumer & Community Banking?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Higher net interest income, lower auto lease depreciation, and a reserve build in Card Services and Home Lending.'

In [ ]:
query = "???"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)